# PyGMTSAR Python Notebook S1A_Stack_CPGF_T173

### Tested on MacOS Catalina (Python 3.9) and Debian 10 (Python 3.7)
### Use the provided init script for Google Cloud Jupyter Notebook on Debian 10:
https://github.com/mobigroup/gmtsar/blob/master/gmtsar/sh/GMTSAR.install.debian10.sh

### I'm a freelancer and that's my free time Open Source project with GPL-3.0 License. If you find it useful you are able to sponsor my projects <a href="https://www.patreon.com/bePatron?u=54500608" data-patreon-widget-type="become-patron-button">Become a Patron!</a><script async src="https://c6.patreon.com/becomePatronButton.bundle.js"></script> or order additional research, development and support on <a href="https://www.upwork.com/freelancers/~01e65e8e7221758623">Upwork</a>

### @ Alexey Pechnikov, Sep, 2021, https://github.com/mobigroup

## Check for GMTSAR installation

#### Just wait 10-30 minutes and restart the notebook if cloud installation is not ready

In [ ]:
count = !ls /usr/local | grep GMTSAR | wc -l
assert count != ['0'], \
    'Please wait until your init script complete on a cloud host or install GMTSAR manually on local host'

## Download and unpack the example, create processing directory

In [ ]:
count = !ls | grep S1A_Stack_CPGF_T173.tar.gz | wc -l
if count == ['0']:
    print ('Downloading the example...')
    !wget -c http://topex.ucsd.edu/gmtsar/tar/S1A_Stack_CPGF_T173.tar.gz
    !tar xvzf S1A_Stack_CPGF_T173.tar.gz -C .

# recreate work directory
!rm -rf raw
!mkdir -p raw

## Define ENV Variables for Jupyter Instance

In [ ]:
import os

# use default GMTSAR installation path
GMTSAR = '/usr/local/GMTSAR'
PATH = os.environ['PATH']

if PATH.find('GMTSAR') == -1:
    PATH = os.environ['PATH'] + f':{GMTSAR}/bin/'
    %env PATH {PATH}
    %env GMTSAR {GMTSAR}

## Install Python Modules

In [ ]:
import sys
!{sys.executable} --version

In [ ]:
!{sys.executable} -m pip install --upgrade pip setuptools wheel > /dev/null
!{sys.executable} -m pip install cartopy==0.19.0.post1 > /dev/null

In [ ]:
!{sys.executable} -m pip install \
    h5py netcdf4 h5netcdf \
    rasterio rioxarray xarray numpy \
    scikit-image scipy sklearn \
    xarray dask distributed zarr \
    pandas geopandas \
    sentineleof elevation \
    matplotlib seaborn geoviews hvplot datashader bokeh \
    xmltodict joblib tqdm 2>&1 > /dev/null

## Load and Setup Python Modules

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
# supress numpy warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# plotting modules
import hvplot.xarray  # noqa
import hvplot.pandas  # noqa
import holoviews as hv
from holoviews import opts
from bokeh.models import FixedTicker
hv.extension('bokeh', 'matplotlib')
#pd.options.plotting.backend = 'holoviews'
pd.options.plotting.backend = 'hvplot'
%pylab inline

In [ ]:
# define Pandas display settings
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

gstiles = hv.Tiles('https://mt1.google.com/vt/lyrs=s&x={X}&y={Y}&z={Z}', name='Google Satellite')
ottiles = hv.Tiles('https://tile.opentopomap.org/{Z}/{X}/{Y}.png', name='Open Topo')

## Load Custom Python Modules

In [ ]:
sys.path.append(os.path.join(os.environ['GMTSAR'],'gmtsar', 'py'))

from PRM import PRM
from SBAS import SBAS

## Define Parameters

In [ ]:
MASTER       = '2015-04-03'
WORKDIR      = 'raw'
DATADIR      = 'raw_orig'
DEMFILE      = 'topo/dem.grd'
BASEDAYS     = 100
BASEMETERS   = 150
CORRLIMIT    = 0.10
DEFOMAX      = 0

## Init SBAS

In [ ]:
sbas = SBAS(DATADIR, DEMFILE, WORKDIR).set_master(MASTER)
sbas.to_dataframe()

### Show Static Plots

In [ ]:
plt.figure(figsize=(12,4))
sbas.get_dem()[::4,::4].plot.imshow(cmap='Blues_r', vmin=0)
plt.scatter(sbas.geoloc()['longitude'], sbas.geoloc()['latitude'], c=sbas.geoloc()['pixel'], cmap='jet')
plt.title('Sentinel1 Frame on DEM plus GCP', fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
sbas.get_dem(geoloc=True)[::4,::4].plot.imshow(cmap='Blues_r', vmin=0)
plt.scatter(sbas.geoloc()['longitude'], sbas.geoloc()['latitude'], c=sbas.geoloc()['pixel'], cmap='jet')
plt.title('Sentinel1 Frame DEM cropped using GCP', fontsize=20)
plt.show()

### Show Interactive Plots

#### Be careful because these plots require more RAM to be visualized

## Stack Images (for a single subswath only)

In [ ]:
sbas.stack_parallel()

## SBAS Baseline

In [ ]:
baseline_pairs = sbas.baseline_pairs(days=BASEDAYS, meters=BASEMETERS)
baseline_pairs

### TODO: Show Plot

## DEM in Radar Coordinates

In [ ]:
%%time

sbas.topo_ra()

### Load Grids

In [ ]:
topo_ra = xr.open_dataarray(f'{WORKDIR}/topo_ra.grd')

### Show Static Plot

In [ ]:
plt.figure(figsize=(8,6))
topo_ra[::4,::4].plot.imshow(cmap='Blues_r', vmin=0)
plt.title('Topography in Radar Coordinates', fontsize=20)
plt.show()

### Show Interactive Plots

#### Be careful because these plots require more RAM to be visualized

## Interferograms

In [ ]:
pairs = baseline_pairs[['ref_date', 'rep_date']]
pairs

In [ ]:
# we can just miss "func" argument when post-processing is not required
# define a postprocessing function for decimation, etc.
decimator = lambda dataarray: dataarray.coarsen({'y': 4, 'x': 4}, boundary='trim').mean()

# default parameters: wavelength=200, psize=32, func=None (no postprocessing required)
sbas.intf_parallel(pairs, wavelength=400, func=decimator)

### Load Grids

In [ ]:
# TODO: add geocode=True/False argument
phasefilts = sbas.open_grids(pairs, 'phasefilt')

### Show Static Plots

In [ ]:
fg = phasefilts.plot.imshow(
    col="pair",
    col_wrap=3, size=4, aspect=1.2,
    vmin=-np.pi, vmax=np.pi, cmap='gist_rainbow_r'
)
fg.set_axis_labels(x_var='Range', y_var='Azimuth')
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('Filtered Phase, [rad]', y=1.05, fontsize=24)
plt.show()

### Show Interactive Plots

#### Be careful because these plots require more RAM to be visualized

### Load Correlation Grids

In [ ]:
corrs = sbas.open_grids(pairs, 'corr')

### Show Static Plots

In [ ]:
fg = corrs.plot.imshow(
    col="pair",
    col_wrap=3, size=4, aspect=1.2,
    clim=(0, 0.8), cmap='gray'
)
fg.set_axis_labels(x_var='Range', y_var='Azimuth')
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('Correlation', y=1.05, fontsize=24)
plt.show()

### Show Interactive Plots

#### Be careful because these plots require more RAM to be visualized

## Unwrapping

In [ ]:
# generate a custom snaphu config file and use it as argument "conf" value
# conf = self.PRM().snaphu_config(defomax=0)

# we can just miss "func" argument when post-processing is not required
# define a post-processing function to crop and interpolate low-coherence areas, etc.
cleaner = lambda corr, unwrap: xr.where(corr>=CORRLIMIT, unwrap, np.nan)
#cleaner = lambda corr, unwrap: sbas.nearest_grid(xr.where(corr>=CORRLIMIT, unwrap, np.nan))

# default parameters: threshold=0.1, conf=None, func=None (no postprocessing required)
sbas.unwrap_parallel(pairs, threshold=CORRLIMIT, func=cleaner)

### Load Grids

In [ ]:
unwraps = sbas.open_grids(pairs, 'unwrap')

### Show Static Plots

In [ ]:
zmin, zmax = np.nanquantile(unwraps, [0.01, 0.99])
fg = unwraps.plot.imshow(
    col="pair",
    col_wrap=3, size=4, aspect=1.2,
    vmin=zmin, vmax=zmax, cmap='jet'
)
fg.set_axis_labels(x_var='Range', y_var='Azimuth')
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('Unwrapped Phase, [rad]', y=1.05, fontsize=24)
plt.show()

### Show Interactive Plots

#### Be careful because these plots require more RAM to be visualized

## LOS Displacement

### Calculate Grids

In [ ]:
los_disp_mm = sbas.los_displacement_mm(unwraps)

### Show Static Plots

In [ ]:
zmin, zmax = np.nanquantile(los_disp_mm, [0.01, 0.99])
fg = los_disp_mm.plot.imshow(
    col="pair",
    col_wrap=3, size=4, aspect=1.2,
    vmin=zmin, vmax=zmax, cmap='jet'
)
fg.set_axis_labels(x_var='Range', y_var='Azimuth')
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('LOS Displacement, [mm]', y=1.05, fontsize=24)
plt.show()

### Show Interactive Plots

#### Be careful because these plots require more RAM to be visualized

### Geocoding

In [ ]:
los_disp_mm_ll = sbas.intf_ra2ll(los_disp_mm)

### Show Static Plots

In [ ]:
zmin, zmax = np.nanquantile(los_disp_mm_ll, [0.01, 0.99])
fg = los_disp_mm_ll.plot.imshow(
    col="pair",
    col_wrap=3, size=4, aspect=1.2,
    vmin=zmin, vmax=zmax, cmap='jet'
)
fg.set_ticks(max_xticks=5, max_yticks=5, fontsize='medium')
fg.fig.suptitle('LOS Displacement in Geographic Coordinates, [mm]', y=1.05, fontsize=24)
plt.show()

## Bonus: Inverted Interferograms

We are able to build reverse-ordered interferograms by a simple hand move

### Load Grids

### Show Static Plots

### Show Interactive Plots

#### Be careful because these plots require more RAM to be visualized

## Bonus: Inverted Interferogram Unwrapping

### Load Grids

### Show Static Plots

### Show Interactive Plots

#### Be careful because these plots require more RAM to be visualized

## Bonus: Interferogram Unwrapping Two-Ways Difference

### Calculate Grids

### Show Static Plots

### Show Interactive Plots

#### Be careful because these plots require more RAM to be visualized

## SBAS Velocity

In [ ]:
sbas.sbas(baseline_pairs, smooth=1)

### Load Grids

In [ ]:
vel = xr.open_dataarray('raw/vel.grd')

### Geocoding

In [ ]:
%%time

vel_ll = sbas.intf_ra2ll(vel)
# interpolate low-coherency areas
vel_filled = sbas.nearest_grid(vel)
vel_filled_ll = sbas.intf_ra2ll(vel_filled)

### Show Static Plots

In [ ]:
%%time

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
zmin, zmax = np.nanquantile(vel, [0, 0.98])
vel.plot(vmin=zmin, vmax=zmax, cmap='jet', ax=ax1)
vel_ll.plot(vmin=zmin, vmax=zmax, cmap='jet', ax=ax2)
vel_filled.plot(vmin=zmin, vmax=zmax, cmap='jet', ax=ax3)
vel_filled_ll.plot(vmin=zmin, vmax=zmax, cmap='jet', ax=ax4)
ax1.set_title('Radar Coordinates', fontsize=20)
ax2.set_title('Geographic Coordinates', fontsize=20)
plt.suptitle('SBAS Velocity, [mm/year]', fontsize=24)
plt.show()

### Show Interactive Plots

#### Be careful because these plots require more RAM to be visualized

#### The plots below do not work on Google Cloud Jupyter (Debian 10 and Python 3.7)

In [ ]:
uname = !uname
if uname == ['Darwin']:
    zmin, zmax = np.nanquantile(vel, [0, 0.98])
    opts_common = {'x':'lon', 'y':'lat', 'geo':True, 'width':320, 'height':280,
                   'cmap':'jet', 'colorbar':True, 'clim':(zmin, zmax)}

    opts1 = {'tiles':gstiles, 'alpha':0.3, 'title':'Google Satellite'}
    opts2 = {'tiles':ottiles, 'alpha':0.3, 'title':'OpenTopoMap'}
    opts3 = {'tiles':False,   'alpha':1.0, 'title':'SBAS Velocity'}

    vel_filled_ll.hvplot(**{**opts_common, **opts1}) + \
    vel_filled_ll.hvplot(**{**opts_common, **opts2}) + \
    vel_filled_ll.hvplot(**{**opts_common, **opts3})